<center><br><br>
    Arkansas Work-Based Learning to Workforce Outcomes <br>
    Applied Data Analytics Training | Spring 2022
    <h1> Creating a Cohort </h1>
    <span style="font-size: 1.5em;">
        <a href='https://www.coleridgeinitiative.org'>Coleridge Initiative</a>
    </span>
    <center>Robert McGough, Joshua Edelmann, Benjamin Feder</center>
</center>

***

Now that you have been introduced to the datasets available for you in this training and have taken a sufficient amount of time to explore potentially relevant variables, you should be ready to begin mapping out the beginnings of your longitudinal analysis. This notebook introduces you to the concept of creating a group or a "cohort" that define the primary population of interest in much research. A cohort is a group of individuals that share a time-based characteristic. Within the focus of the training program, your cohort will consist of a population of registered apprenticeship program completers. Once created, cohorts may then be used to link to other data sources, such as employment data.

In creating a cohort, we will walk you through some of the deicsions that need to be made when defining a population of apprenticeship completers, such as filters to define completers and the time frame they completed. After doing so, we will construct descriptive statistics to better understand the population of interest.

## 1. Getting Started

In [ ]:
options(warn = -1)                   # switches warnings off

suppressMessages(library(odbc))      # allows R to connect with the database
suppressMessages(library(tidyverse)) # useful for data manipulation and visualization
suppressMessages(library(scales))    # to calculate percentages, graphing
suppressMessages(library(lubridate)) # for easy working with dates 

options(warn = 0)                    # switches warnings on 
options(scipen=999)                  # prevents scientific notation

In [ ]:
# server connection
con <- DBI::dbConnect(odbc::odbc(),
                     Driver = "SQL Server",
                     Server = "msssql01.c7bdq4o2yhxo.us-gov-west-1.rds.amazonaws.com",
                     Trusted_Connection = "True")

## 2. Defining a Cohort
In this notebook, we will explore a cohort of individuals who participated in a registered apprenticeship program in the state of Arkansas and completed their training sometime between 2015 and 2017. In your own projects, you might want to explore alternative constructions, such as individuals who entered their apprenticeship program during a particular time window. Recall that we only have longitudinal wage and education data available for those in Arkansas.

We will start our cohort exploration by defining this basic analytical frame from the complete **rapids_apprentice** and **rapids_program** tables in the **tr_ar_2022** database. We also include the NAICS descriptions for each NAICS code. These descriptions are stored in the **ds_ar_dws** database and in the table **NAICS_National_Industry**. It may take a few seconds to read the data. 

> Note: It is common practice to only include the necessary variables within your cohort definition. You can narrow down the list of variables through a combination of an exploratory data analysis and familiarizing yourself with the data dictionary. Please do not try to read in any of the complete **rapids_** tables without identifying a small subset of columns. You will receive an error if you attempt to read in every column into R.

In [ ]:
# initial cohort query
# recall that commenting inside a SQL query is done by "--"
query <- "
SELECT
RP.psnumber,
RP.progname,
RP.standardstype,
RA.apprnumber,
RA.naicscode,
NNI.Name,
RA.occupationtitle,
RA.onetsoccode,
RA.apprstatus,
RA.progstate,
RA.progzip5,
RA.county,
RA.termlengthmin,
RA.gender,
RA.race,
RA.ethnicity,
RA.vetstatind,
RA.disabled,
RA.ageatstart,
RA.exitwagedt
FROM ds_public_1.dbo.rapids_apprentice RA
JOIN ds_public_1.dbo.rapids_program RP ON (RP.psnumber =RA.psnumber)
LEFT JOIN ds_ar_dws.dbo.NAICS_National_Industry NNI ON (NNI.Code = RA.naicscode)
WHERE RA.progstate='AR' --RESTRICT TO ARKANSAS PARTICIPANTS ONLY
AND YEAR(RA.exitwagedt) BETWEEN 2015 AND 2017 --RESTRICT TO THESE YEARS
AND RA.apprstatus = 'CO' -- RESTRICT TO COMPLETION STATUS
"

rapids_cohort <- dbGetQuery(con, query)

# view the first 6 observations
head(rapids_cohort)

Let's see if we have any duplicate apprentice numbers in our cohort. We can do this by comparing the count of number of rows to the distinct count of apprentice numbers.

In [ ]:
# find number of rows and unique apprentice number in our cohort
rapids_cohort %>%
    summarize(
        n_rows = n(),
        n_appr = n_distinct(apprnumber))

We have the same number of rows and distinct apprenticeship numbers. This is telling us that there are no duplicate values. We will confirm this a little later on.

To get a better understanding of the **apprstatus** variable, which we used to identify apprenticeship completers, we can read in the variable **apprstatusreason** to see the distinct notes written for the **CO** status. 

In [ ]:
# see apprenticeship status notes for completers
qry <- "
SELECT distinct(apprstatusreason) 
FROM ds_public_1.dbo.rapids_apprentice ra 
WHERE apprstatus = 'CO';
"
dbGetQuery(con, qry)

An immediate concern is a data quality issue. The **CO** status corresponds to apprenticeship completion, but the reason given does not always align with that status. This is something to keep in mind as we move forward.

> Note: The most common **apprstatusreason** values for completers correspond to actual apprenticeship completion.

### Multiple Completions

Given that the count of unique apprentice number values is the same count as number of rows, we know that there are not any duplicate values in our cohort. We can double-check this though.

In [ ]:
# find example of someone showing up multiple times
# use pull to isolate the contents of the variable (individual aapprnumber) and save as an object
apprnum_dup <- rapids_cohort %>%
    count(apprnumber) %>%
    arrange(desc(n)) %>%
    head()

apprnum_dup

We can confirm that we do not have duplicate apprentice number values in our cohort. If we did, we would need to figure out how to account for those with multiple completions.

## 3. Understanding the Cohort

After creating your cohort, you may, understandably, want to transition right into record linkage and begin your longitudinal analysis. However, it is important to pause to learn more about your cohort. Taking this step will allow you to contextualize future findings and provide additional verification of the cohort construction. In this process, as in EDA, it is highly encouraged to take note and discuss any missingness and abnormal values.

In this notebook, we will calculate descriptive statistics on the following:

- Gender
- County
- NAICS
- Intersection of Gender and NAICS

As you're working through this section, think about subgroup and measurement distributions you may want to implement in your project.

### Gender Distribution

An important demographic breakdown worth considering in your analytical framework is the gender distribution of the cohort. Within the RAPIDS data, there is the **gender** variable. Let's look at the distribution of this variable.

In [ ]:
# gender distribution
rapids_cohort %>%
    count(gender)

We can see that there are significantly more male apprenticeship completers than female ones.

### County Distribution

Disparities exist between rural and urban counties. Individuals living in rural areas may not have access to the same resources as those living in urban areas. We can see the breakdown of our cohort by counties.

In [ ]:
# most common counties 
rapids_cohort %>% 
    count(county) %>%
    arrange(desc(n)) %>%
    head()

We see that Pulaski County has the highest count of participants. This is not surprising since this county is the most populous county in the state and the capital is located in this county.

> Note: You can use the public Urban-Rural Continuum data to identify urban and rural counties.

### NAICS Code

To explore our cohort even further, we can focus on the **naicscode** variable, which tracks the corresponding industry code (NAICS) of the apprenticeship program. We will include the **NAME** variable, as it describes each of the NAICS codes.

In [ ]:
# most common industries 
rapids_cohort %>%
    count(naicscode, Name) %>%
    arrange(desc(n)) %>%
    head()

Most of our cohort completed their apprenticeship in electrical contractors and other wiring installation contractors, and commercial and institutional building construction jobs.

> Note: Due to cohort size and granularity considerations, it may be more useful to use NAICS codes with less than six digits, as they represent broader industries.

### Intersection of Gender and NAICS

Another important factor to consider is the intersection of gender and NAICS codes. We already know that more males completed apprenticeships, but we don't know if they completed apprenticeships in similar industries.

In [ ]:
# see most common gender/naics by gender combinations
rapids_cohort %>%
    count(gender, naicscode, Name) %>%
    group_by(gender) %>%
    arrange(desc(n)) %>%
    slice(1:5)

We can see that while both genders tended to complete apprenticeships in either construction or contracting, the order changed for the most likely apprenticeships by gender.

## 4. Saving as a Permanent Table

Since we performed the cohort filtering in R, we can save the final cohort definition as a csv file. However, we can also save the file as a permanent table in R, which will allow us to leverage SQL's join functionality to link the cohort to large tables that are unable to be read into R due to memory constraints. The following code blocks demonstrate how to convert an R data frame to a permanent table, creating the table **nb_cohort** in the AR training workspace database **tr_ar_2022** from the data frame **rapids_cohort**.

> Saving your cohort as a permanent table is *highly* recommended. When doing so, please save your table with the prefix **team#** (ex. **team1_cohort**) to differentiate between tables for each project.

    qry <- "use tr_ar_2022;"
    DBI::dbExecute(con, qry)

    DBI::dbWriteTable(
        conn = con,
        name = DBI::SQL("dbo.nb_cohort"), 
        value = rapids_cohort
    )

> Note: If you run this code, you will get an error because a table with the name `nb_cohort` already exists in the `tr_ar_2022` database.

## References


TDC Creating a Cohort (link to come)